In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [2]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_vnd.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.isocalendar().week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
stock = 'VND'
df.tail(3)


,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,
VND,20211208,75.6,76.5,74.5,75.9,4521100,2021-12-08,49,2021,2021-49
VND,20211209,75.4,79.9,75.1,79.9,5493000,2021-12-09,49,2021,2021-49
VND,20211210,79.5,80.7,78.0,79.0,4585800,2021-12-10,49,2021,2021-49


In [3]:
df = df.rename(columns={'<CloseFixed>':'<Close>'})
df = df[['<Close>','datetime','<Volume>','Period','Year_no']]

In [4]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name=stock))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [5]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart')

In [6]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
VND,75.9,2021-12-08,4521100,2021-49,2021
VND,79.9,2021-12-09,5493000,2021-49,2021
VND,79.0,2021-12-10,4585800,2021-49,2021


In [7]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
VND,7.4814,2010-03-31,846200,2010-13,2010,3.571725
VND,7.5846,2010-04-01,412900,2010-13,2010,1.379421
VND,7.5158,2010-04-02,569400,2010-13,2010,-0.907101


In [8]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [9]:
df['Daily_return'].describe()

count    2922.000000
mean        0.120787
std         2.793815
min        -9.894737
25%        -1.331831
50%         0.000000
75%         1.504292
max         9.999572
Name: Daily_return, dtype: float64

In [10]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [11]:
df['<Close>'].describe()

count    2922.000000
mean        9.153956
std        10.839681
min         1.936100
25%         4.418800
50%         5.737000
75%         9.289625
max        81.400000
Name: <Close>, dtype: float64

In [12]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [13]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                      mode ='lines', name='Span_7_Rolling_vol'))

fig.add_trace(go.Scatter(x=df.datetime, y=df['<Close>'], 
                        mode='lines+markers', name='Price'))

In [14]:
df['Volumn'].head(30).describe()

count    30.000000
mean      0.767157
std       0.224264
min       0.388200
25%       0.602550
50%       0.751000
75%       0.842375
max       1.534400
Name: Volumn, dtype: float64

In [15]:
df['Volumn'].head(10).describe()

count    10.000000
mean      0.835390
std       0.307786
min       0.412900
25%       0.730725
50%       0.784900
75%       0.838725
max       1.534400
Name: Volumn, dtype: float64

In [16]:
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return,Volumn
<Ticker>,,,,,,,
VND,7.4814,2010-03-31,846200,2010-13,2010,3.571725,0.8462
VND,7.5846,2010-04-01,412900,2010-13,2010,1.379421,0.4129
VND,7.5158,2010-04-02,569400,2010-13,2010,-0.907101,0.5694
